In [1]:
! pip install -q python-dotenv
! pip install -q neo4j
! pip install -q langchain
! pip install -q langchain-openai
! pip install -q tiktoken


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import dotenv
import os

dotenv.load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("URL")
os.environ["NEO4J_USERNAME"]= os.getenv("USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("PASSWORD2")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAIKEY")

In [3]:
from langchain_community.graphs import Neo4jGraph


In [4]:
graphDB = Neo4jGraph()

In [5]:
#
# Documentation: https://platform.openai.com/docs/guides/embeddings/use-cases
#


from openai import OpenAI

client =OpenAI()

def get_embedding(text, model = "text-embedding-3-small"):

    text = text.replace("\n"," ")

    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [6]:
graphDB.query("SHOW INDEXES")

[{'id': 3,
  'name': 'accreditation-index',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 1,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 15, 8, 39, 59, 51000000, tzinfo=<UTC>),
  'readCount': 44},
 {'id': 2,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

In [7]:
semantic_index_query = """

CALL db.index.vector.queryNodes('accreditation-index', 2, $inputVector)
YIELD node AS responseNode, score

RETURN responseNode.text, score 
"""

In [8]:
query_text = "what are the marketing courses"
query_vector = get_embedding(query_text)

query_result = graphDB.query(semantic_index_query, 
                                params={
                                    "inputVector":query_vector
                                })

In [9]:
query_result

[{'responseNode.text': 'BBA Objective 2 .3: Demonstrate basic functional abilities across core business subjects  \nBecause this goal covers a large area of knowledge, two courses taken by all business majors w as \nassessed in addition to using the ETS exam as an overall assessment.   \n LENB 3135 was used to assess how well students could discuss the laws that relate to contracts, \nincluding the UCC. In 14- 15, the fall semester students met or exceed the target at an 80% rate while the \nspring semester seemed to be an anomaly as only 62% met or exceeded the target. The same assessment was used in 15- 16. For fall semester, 74% met or exceeded expectations, and in spring 16, \n72% met or exceeded expectations . \n \nMKTG 3161 was used to assess how well students identify key marketing concepts and apply them to \nreal-world business problems. In 13- 14, 68% met the target, but another 16% only missed by 1 question. \nThis assessment was  continued with continued emphasis on repetit

In [11]:
#
# Standard QA
#
from langchain_community.vectorstores import Neo4jVector

VECTOR_INDEX_NAME = 'accreditiation-index'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY='embedding'


In [ ]:
#https://python.langchain.com/docs/modules/data_connection/retrievers/MultiQueryRetriever/
